In [41]:
from langchain_groq import ChatGroq

In [42]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama3-8b-8192"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to walk on the Moon was Neil Armstrong, an American astronaut who stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.


In [43]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/digital-content-designer-ii-global-converse/job/R-62533")
page_data = loader.load().pop().page_content
print(page_data)






















Digital Content Designer II Global - Converse










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return 

In [44]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [45]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Digital Content Designer II Global - Converse',
  'experience': "2-4+ years' experience in digital design for large retail e-commerce or key digital agency experience specific to the streetwear footwear/apparel industry.",
  'skills': 'Adobe Suite, Frame.io, and other post-production software, Understanding of animation, video editing, photo retouching—Skills in Abode After Effects a plus',
  'description': 'Working under the direction of the Manger, Global Digital Content, you will concept all aspects of our content creation which fuel our digital experiences and wholesale channels (focusing on, but not limited to, converse.com and converse.net experiences, emails and both paid and organic social and other forms of paid media ) In this role you will own the concepting through post-production and final finished assets of photography You will handle product & lifestyle content—experience being the lead art director on set, working with product, talent, photo/videographers, pr

In [46]:
type(json_res)

list

In [47]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [48]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [49]:
# Assuming json_res is like: [{'skills': ['Python', 'Java']}, {'name': 'Alice'}]
if isinstance(json_res, list) and len(json_res) > 0 and isinstance(json_res[0], dict) and 'skills' in json_res[0]:
    skills_list = json_res[0]['skills']
    links = collection.query(query_texts=skills_list, n_results=2).get('metadatas', [])
else:
    print("Error: 'skills' dictionary not found at expected location in json_res.")

In [50]:
json_res

[{'role': 'Digital Content Designer II Global - Converse',
  'experience': "2-4+ years' experience in digital design for large retail e-commerce or key digital agency experience specific to the streetwear footwear/apparel industry.",
  'skills': 'Adobe Suite, Frame.io, and other post-production software, Understanding of animation, video editing, photo retouching—Skills in Abode After Effects a plus',
  'description': 'Working under the direction of the Manger, Global Digital Content, you will concept all aspects of our content creation which fuel our digital experiences and wholesale channels (focusing on, but not limited to, converse.com and converse.net experiences, emails and both paid and organic social and other forms of paid media ) In this role you will own the concepting through post-production and final finished assets of photography You will handle product & lifestyle content—experience being the lead art director on set, working with product, talent, photo/videographers, pr

In [51]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(json_res), "link_list": links})
print(res.content)

Subject: Enhance Your Digital Content Creation with AtliQ's Expertise

Dear [Hiring Manager's Name],

I came across the job description for the Digital Content Designer II Global - Converse role and was impressed by the scope of responsibilities. As a business development executive at AtliQ, I'd like to introduce our company's capabilities in fulfilling your content creation needs.

AtliQ is a leading AI & Software Consulting company that specializes in streamlining business processes through automated tools. Our expertise lies in empowering enterprises with tailored solutions, resulting in scalability, process optimization, cost reduction, and heightened overall efficiency.

I'd like to highlight that our portfolio includes projects that showcase our ability to integrate cutting-edge technology with creative vision. For instance, our work on [link: https://example.com/ios-ar-portfolio] demonstrates our expertise in augmented reality (AR) and its applications in digital content creatio